In [1]:
import pandas as pd

In [3]:
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
import pandas as pd
wine = pd.read_csv(csvurl)
wine.drop(wine.columns[0], axis="columns", inplace=True)
wine.head()
print(wine)

         country                                        description  \
0          Italy  Aromas include tropical fruit, broom, brimston...   
1       Portugal  This is ripe and fruity, a wine that is smooth...   
2             US  Tart and snappy, the flavors of lime flesh and...   
3             US  Pineapple rind, lemon pith and orange blossom ...   
4             US  Much like the regular bottling from 2012, this...   
...          ...                                                ...   
65494     France  Made from young vines from the Vaulorent porti...   
65495  Australia  This is a big, fat, almost sweet-tasting Caber...   
65496         US  Much improved over the unripe 2005, Fritz's 20...   
65497         US  This wine wears its 15.8% alcohol better than ...   
65498      Spain  A unique take on Manzanilla Sherry, which is o...   

                              designation  points  price           province  \
0                            Vulkà Bianco      87    NaN  Sicily & S

In [4]:
mean_points = wine['points'].mean()

print("Mean of points column:", mean_points)

Mean of points column: 88.43403716087269


In [5]:
unique_countries = wine['country'].nunique()

print("Number of unique countries:", unique_countries)

Number of unique countries: 41


In [7]:
country_counts = wine['country'].value_counts(ascending=True)

print(country_counts)

country
Bosnia and Herzegovina        1
Armenia                       1
Slovakia                      1
Switzerland                   4
Luxembourg                    4
India                         4
Ukraine                       5
Macedonia                     6
Czech Republic                6
Cyprus                        6
Serbia                        7
Peru                          8
Morocco                      11
Lebanon                      20
Moldova                      30
Brazil                       31
Mexico                       31
England                      36
Georgia                      37
Slovenia                     39
Turkey                       43
Croatia                      44
Uruguay                      61
Hungary                      61
Romania                      67
Bulgaria                     68
Canada                      108
Greece                      242
Israel                      259
New Zealand                 733
South Africa                737


In [9]:
average_price = wine['price'].mean(skipna=True)

# Create the adjusted_price column by subtracting the average price, ignoring NaN values
wine['adjusted_price'] = wine['price'] - average_price

print(wine[['price', 'adjusted_price']])

       price  adjusted_price
0        NaN             NaN
1       15.0      -20.232932
2       14.0      -21.232932
3       13.0      -22.232932
4       65.0       29.767068
...      ...             ...
65494   45.0        9.767068
65495   22.0      -13.232932
65496   20.0      -15.232932
65497   31.0       -4.232932
65498   10.0      -25.232932

[65499 rows x 2 columns]


In [10]:
wine['points_to_price_ratio'] = wine['points'] / wine['price']

# Find the row with the maximum points-to-price ratio
max_ratio_row = wine.loc[wine['points_to_price_ratio'].idxmax()]

# Print the title of the wine with the highest points-to-price ratio
print("Wine with highest points-to-price ratio:", max_ratio_row['title'])

Wine with highest points-to-price ratio: Bandit NV Merlot (California)


In [12]:

# Count wines with the word "tart" (whole word, not case sensitive)
tart_count = wine['description'].str.contains(r'\btart\b', case=False, na=False).sum()

# Count wines with the word "berries" (whole word, not case sensitive)
berries_count = wine['description'].str.contains(r'\bberries\b', case=False, na=False).sum()

# Create the flavor_counts series
flavor_counts = pd.Series([tart_count, berries_count], index=["Tart", "Berries"])

print(flavor_counts)

Tart       3086
Berries    1192
dtype: int64


In [14]:
def get_star_rating(row):
    if row['country'] == 'France':  # Special case for France
        return 3
    elif row['points'] >= 90:
        return 3
    elif row['points'] >= 80:
        return 2
    else:
        return 1

# Apply the star rating logic based on points and country
wine['star_ratings'] = wine.apply(get_star_rating, axis=1)

# Show the first few rows of the dataframe with the new column
print(wine[['points', 'country', 'star_ratings']].head(10))

   points   country  star_ratings
0      87     Italy             2
1      87  Portugal             2
2      87        US             2
3      87        US             2
4      87        US             2
5      87     Spain             2
6      87     Italy             2
7      87    France             3
8      87   Germany             2
9      87    France             3


In [16]:
reviewer_counts = wine['taster_twitter_handle'].value_counts()

# Show the Series containing the count of reviews for each taster
print(reviewer_counts)

taster_twitter_handle
@vossroger          13045
@wineschach          7752
@kerinokeefe         5313
@paulgwine           4851
@vboone              4696
@mattkettmann        3035
@JoeCz               2605
@wawinereport        2358
@gordone_cellars     2032
@AnneInVino          1769
@laurbuzz             938
@suskostrzewa         593
@worldwineguys        465
@bkfiona               11
@winewchristina         4
Name: count, dtype: int64


In [17]:
country_variety_counts = wine.groupby(['country', 'variety']).size()

# Sort the values in descending order
country_variety_counts = country_variety_counts.sort_values(ascending=False)

# Show the Series with the MultiIndex and counts
print(country_variety_counts)

country  variety                 
US       Pinot Noir                  4918
         Cabernet Sauvignon          3649
         Chardonnay                  3412
France   Bordeaux-style Red Blend    2380
Italy    Red Blend                   1870
                                     ... 
         Torbato                        1
         Vespaiolo                      1
         Vespolina                      1
         Vitovska                       1
Uruguay  Tempranillo-Tannat             1
Length: 1304, dtype: int64


In [18]:
# Assuming you have a DataFrame df with 'taster_name' and 'points' columns
# Group by taster_name and calculate the average points for each reviewer
average_scores_by_reviewer = wine.groupby('taster_name')['points'].mean()

# Show the Series with reviewers and their average scores
print(average_scores_by_reviewer)

taster_name
Alexander Peartree    86.014286
Anna Lee C. Iijima    88.380506
Anne Krebiehl MW      90.587903
Carrie Dykes          86.644444
Christina Pickard     89.500000
Fiona Adams           87.090909
Jeff Jenssen          88.273504
Jim Gordon            88.604331
Joe Czerwinski        88.519770
Kerin O’Keefe         88.827969
Lauren Buzzeo         87.831557
Matt Kettmann         90.021087
Michael Schachner     86.904541
Mike DeSimone         89.030303
Paul Gregutt          89.095032
Roger Voss            88.678957
Sean P. Sullivan      88.666243
Susan Kostrzewa       86.408094
Virginie Boone        89.229557
Name: points, dtype: float64
